# 1 Load Data
## 1.1 Import Packages
I will work with this dataset as a pandas dataframe, so I will need the pandas package. Additionally, I will visualize this as a choropleth, so I will need the folium package to map the values. 

In [1]:
import pandas as pd
import folium

## 1.2 Import Data
There are two CSVs I will need for this project. The first is the opioid prescription data itself, and the second is the county demographic data, which I scraped from Wikipedia. 

In [2]:
#read in the two csv files: ut_op is the opioid prescription data, and ut_counties is the county population data
ut_op = pd.read_csv("Utah_Opioid_Prescription_Data_by_City_Zip___County_2015_2016.csv")
ut_counties = pd.read_csv("ut_county_pops.csv")

# 2 Data Analysis
## 2.1 Data Grouping
The data are separated out by individual healthcare provider, but I want county-level data. To get this, I will use the groupby() function, and group the data by county, getting the sum of prescriptions for each group.

In [3]:
#find the total number of opioid prescriptions for each county using groupby method
cnty_op_claim = ut_op.groupby('County').OpioidClaimCount.sum()

## 2.2 Data Merging
Now that the prescription data are aggregated by county, we can merge it with the county demographic data on each of their conty columns.

In [4]:
#merge the county level opioid data with the county population data. merge on county name.
cnty_op_claim = pd.merge(cnty_op_claim, ut_counties, how = 'left', on = 'County')

## 2.3 Calucated Columns
To get a rate that can be compared between counties, I will create a new column, "op_scrip_rate", which will simply be the total opioid claim count divided by that population of that county. 

In [5]:
#create a new column that shows the number of opioid perscriptions per person in the county
cnty_op_claim['op_scrip_rate'] = cnty_op_claim['OpioidClaimCount']/cnty_op_claim['Population']

## 2.4 Data Preparation
To make it so the data aligns with the geojson file I am using for the map, I need to change the "County" column to "NAME", as well as change all the strings in the dataframe to upper case.

In [6]:
#rename the county column so it matches the attribute in the 
#geojson file, and change all strings in the dataframe to upper case
cnty_op_claim = cnty_op_claim.rename(columns={'County' : 'NAME'})
cnty_op_claim.columns = map(lambda x: str(x).upper(), cnty_op_claim.columns)
cnty_op_claim['NAME'] = cnty_op_claim['NAME'].str.upper()

# 3 Visualization
I will also use Tableau to visualize this dataset, but for the purposes of completeness (and reproducibility) I will also export this in a format that is open source, using teh folium package.

In [7]:
#make a choropleth map where color is determined based on op_scrip_rate
#use geojson file for county shape data
county_map = '051acb66-8e4b-4c24-9da8-057bfdb69dcf202041-1-1ikt18k.0n4a.geojson'

#establish features of final map (center and zoom)
m = folium.Map(location=[40,-111], zoom_start=7)

#create choropleth
m.choropleth(
 geo_data=county_map,
 name='choropleth',
 data=cnty_op_claim,
 columns=['NAME', 'OP_SCRIP_RATE'],
 key_on='feature.properties.NAME',
 fill_color='BuPu',
 fill_opacity=0.7,
 line_opacity=0.5,
 legend_name='Opioid Prescriptions Per Person (2015-2016)'
)

#save choropleth as html file
m.save('UT_op_choropleth.html')

C:\Users\dalli\anaconda3\envs\Python and R Main\lib\site-packages\folium\folium.py:411: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
